In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'mimic-432606'
client = bigquery.Client(project=project_id)
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# Generic Sepsis Affected Population Check

**Tables:**
--------------------------------------------
*mimiciv_hosp.admission*:  Patient admission information

subject_id; hadm_id; admittime; dischtime;

--------------------------------------------
*mimiciv_hosp.d_icd_diagnoses*: ICD-code mapping

icd_code; icd_version; long_title

--------------------------------------------
*mimiciv_hosp.patients*: Patients' anchor age

subject_id; gender; anchor_age;

--------------------------------------------
*mimiciv_hosp.d_labitems*: Lab tests the hospital does

itemid; label; fluid; category

--------------------------------------------
*mimiciv_hosp.diagnosis_icd*: Patients' diagnosis by ICD-code during an admission

subject_id; hadm_id; seq_num; icd_code; icd_version

--------------------------------------------
*mimiciv_hosp.labevents*: Patients' labtest data by time

labevent_id; subject_id; hadm_id; charttime; value; valuenum; valueuom

--------------------------------------------
*mimiciv_icu.icustays*: Patients ICU stay information

subject_id; hadm_id; stay_id; intime; outtime; los

--------------------------------------------
*mimiciv_icu.outputevents*: Patients body output at ICU stay

subject_id; hadm_id; stay_id; charttime; itemid; value; valueuom


--------------------------------------------

Sepsis code included here:
* R6520: Severe Sepsis
* R6521: Severe Sepsis with septic shock
* 99591: Sepsis
* 99592: Severe Sepsis
* 78552: Close to R6521. Severe Sepsis

## Briefly checking sepsis patients

In [4]:
# Check distinct number of patients who have been diagnosed with any type of Sepsis
sepsis_check_df_1 = run_query("""
SELECT
    DISTINCT d.subject_id,   /* Find distinct count on Subject ID */
FROM
    `physionet-data.mimiciv_hosp.diagnoses_icd`AS d  /* Check patients who have been diagnosed with sepsis in the diagnose table */
WHERE
    d.icd_code = 'R6520' OR d.icd_code = '99591' OR d.icd_code = '99592' OR d.icd_code = '78552' OR d.icd_code = 'R6521';

""")
sepsis_check_df_1
# There are in total 9997 patients who have been diagnosed with Sepsis throughout their medical history

,subject_id
0,10004401
1,10018081
2,10253057
3,10290812
4,10368327
...,...
9992,15229355
9993,16089043
9994,18296400
9995,18412168


In [5]:
# Check total number of admissions of patients who have been diagnosed with any type of Sepsis
sepsis_check_df_2 = run_query("""
SELECT
    DISTINCT d.hadm_id,   /* Find distinct count on the admission id */
FROM
    `physionet-data.mimiciv_hosp.diagnoses_icd`AS d  /* Check patients who have been diagnosed with sepsis in the diagnose table */
WHERE
    d.icd_code = 'R6520' OR d.icd_code = '99591' OR d.icd_code = '99592' OR d.icd_code = '78552' OR d.icd_code = 'R6521';

""")
sepsis_check_df_2

# There are in total 9997 patients who have been diagnosed with Sepsis throughout their medical history

,hadm_id
0,23920883
1,26488315
2,21027282
3,29304348
4,25920662
...,...
11972,21233372
11973,29949151
11974,21737428
11975,23501637


In [6]:
# Find patients who
#                   was diagnosed with any type of Sepsis
#                   was admitted into the ICU and stayed longer than 48 hours

# Admission into the ICU are ordered by chronological order. The first admission into the ICU can be identified
# This did not consider patient age and admission type
sepsis_affected_ICU_patients_df = run_query("""
  SELECT
      i.subject_id,   /* Find distinct count on the admission id */
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  FROM
      `physionet-data.mimiciv_icu.icustays` AS i
  JOIN
      `physionet-data.mimiciv_derived.sepsis3` as s
  ON
      i.stay_id = s.stay_id AND i.subject_id = s.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.patients` AS p /* Patient demographics */
  ON
      i.subject_id = p.subject_id
  WHERE
      s.sepsis3 = true AND i.los >= 2.0
  GROUP BY
      i.subject_id,
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  ORDER BY
      i.subject_id, i.hadm_id, i.intime;
""")
sepsis_affected_ICU_patients_df

,subject_id,hadm_id,stay_id,intime,outtime,los
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817
1,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912
2,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447
3,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512
4,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843
...,...,...,...,...,...,...
21404,19998591,24349193,31144045,2185-07-16 18:48:18,2185-07-26 18:27:01,9.985220
21405,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646
21406,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715
21407,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370


In [7]:
# Consider Patient Age this time.
# Patient age has to be greater than 18 years old and smaller than 90 years old
# Exclude patients who stays less than 2 days
sepsis_affected_ICU_patients_df = run_query("""
  SELECT
      i.subject_id,   /* Find distinct count on the admission id */
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  FROM
      `physionet-data.mimiciv_icu.icustays` AS i
  JOIN
      `physionet-data.mimiciv_derived.sepsis3` as s
  ON
      i.stay_id = s.stay_id AND i.subject_id = s.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.patients` AS p /* Patient demographics */
  ON
      i.subject_id = p.subject_id
  WHERE
      s.sepsis3 = true AND i.los >= 2.0 AND p.anchor_age >= 18 AND p.anchor_age < 90
  GROUP BY
      i.subject_id,
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  ORDER BY
      i.subject_id, i.hadm_id, i.intime;
""")
sepsis_affected_ICU_patients_df

,subject_id,hadm_id,stay_id,intime,outtime,los
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817
1,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912
2,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447
3,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512
4,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843
...,...,...,...,...,...,...
20662,19998591,24349193,31144045,2185-07-16 18:48:18,2185-07-26 18:27:01,9.985220
20663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646
20664,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715
20665,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370


In [8]:
# Consider Patient Age this time.
# Patient age has to be greater than 18 years old and smaller than 90 years old
# Retain patients who stays less than 2 days in the ICU and later drop them.
sepsis_affected_ICU_patients_df = run_query("""
  SELECT
      i.subject_id,   /* Find distinct count on the admission id */
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  FROM
      `physionet-data.mimiciv_icu.icustays` AS i
  JOIN
      `physionet-data.mimiciv_derived.sepsis3` as s
  ON
      i.stay_id = s.stay_id AND i.subject_id = s.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.patients` AS p /* Patient demographics */
  ON
      i.subject_id = p.subject_id
  WHERE
      s.sepsis3 = true AND p.anchor_age >= 18 AND p.anchor_age < 90
  GROUP BY
      i.subject_id,
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  ORDER BY
      i.subject_id, i.hadm_id, i.intime;
""")
sepsis_affected_ICU_patients_df

,subject_id,hadm_id,stay_id,intime,outtime,los
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352
2,10002155,20345487,32358465,2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447
...,...,...,...,...,...,...
31674,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794
31675,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715
31676,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
31677,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741


In [9]:
# Consider Patient Age this time.
# Patient age has to be greater than 18 years old and smaller than 90 years old
# Include paitents all ICD Codes
# Retain patients who stays less than 2 days in the ICU and later drop them.
sepsis_affected_ICU_patients_and_comorbidity_df = run_query("""

  SELECT
      i.subject_id,   /* Subject ID */
      i.hadm_id,      /* Admission ID */
      i.stay_id,      /* ICU Stay ID */
      i.intime,       /* ICU Admission Time */
      i.outtime,      /* ICU Discharge Time */
      i.los,          /* Length of Stay in ICU */
      STRING_AGG(d.icd_code, ', ') AS merged_icd_codes  /* Concatenate all ICD codes for the admission */
  FROM
      `physionet-data.mimiciv_icu.icustays` AS i
  JOIN
      `physionet-data.mimiciv_derived.sepsis3` AS s
  ON
      i.stay_id = s.stay_id AND i.subject_id = s.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.patients` AS p /* Patient demographics */
  ON
      i.subject_id = p.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.diagnoses_icd` AS d /* All diagnoses for the same admission */
  ON
      i.subject_id = d.subject_id AND i.hadm_id = d.hadm_id
  WHERE
      s.sepsis3 = true AND p.anchor_age >= 18 AND p.anchor_age < 90
  GROUP BY
      i.subject_id, i.hadm_id, i.stay_id, i.intime, i.outtime, i.los
  ORDER BY
      i.subject_id, i.hadm_id, i.intime;

""")
sepsis_affected_ICU_patients_and_comorbidity_df

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J..."
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,"41401, 42832, 5180, 4280, 4139, 4400, 4019, 34..."
2,10002155,20345487,32358465,2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576,"1628, 486, 51884, 5781, 2851, 2761, 49121, 414..."
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2..."
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4..."
...,...,...,...,...,...,...,...
31664,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794,"5780, 5723, 2761, 5771, 5712, 45621, 2768, 303..."
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27..."
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7..."
31667,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,"486, 5849, 2760, 5070, 33182, 29410, 78720, 58..."


In [10]:
# Check if they have sort of acute kidney disease
renal_failure_related_icd_codes = set(['5845', '5849', 'N170', 'N171', 'N172', 'N178', 'N179'])

def check_for_aki(codes):
    code_list = {code.strip() for code in codes.split(',')}
    for item in code_list:
      if item in renal_failure_related_icd_codes:
        return '1'
    return '0'

sepsis_affected_ICU_patients_and_comorbidity_df['any_AKI'] = sepsis_affected_ICU_patients_and_comorbidity_df['merged_icd_codes'].apply(check_for_aki)
sepsis_affected_ICU_patients_and_comorbidity_df

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,"41401, 42832, 5180, 4280, 4139, 4400, 4019, 34...",0
2,10002155,20345487,32358465,2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576,"1628, 486, 51884, 5781, 2851, 2761, 49121, 414...",0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0
...,...,...,...,...,...,...,...,...
31664,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794,"5780, 5723, 2761, 5771, 5712, 45621, 2768, 303...",0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0
31667,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,"486, 5849, 2760, 5070, 33182, 29410, 78720, 58...",1


# Pre-filter
Select patients who have been diagnosed with AKI. This step **does not** involve judging the patients' condition for the first 24 hours.

There are two cases when it comes to patient selection.

--------------------------------------------
The first case is a strict selection:

* If a patient is diagnosed with any type of AKI during an admission. Only the first ICU stay of that admission counts. All subsequent admissions are discarded.

* If a patient is not diagnosed any type of AKI. All the ICU stays within that admission count.

---------------------------------------------
The second case is a bit loose:

* If a patient is diagnosed with any type of AKI. Only the first ICU stay of the admission counts, but we do not discard the subsequent admissions

* If a patient is not diagnosed with any type of AKI. All the ICU stays within that admission count.

---------------------------------------------

## strict case

In [11]:
row_list = []  # For fast computation, create an empty list to store rows

diagnosed = False  # Default condition is a patient is not diagnosed with AKI
previous_subject_id = 0  # Starting with 0

for index, row in sepsis_affected_ICU_patients_and_comorbidity_df.iterrows():  # Check for every row of the dataframe

  current_subject_id = row['subject_id']  # Get the current subject ID

  if previous_subject_id == current_subject_id and diagnosed:  # Same patient and diagnosed AKI previously.
    continue  # Ignore that patient

  if previous_subject_id != current_subject_id:  # Encounter a new patient
    diagnosed = False  # Refresh the diagnosis

  if row['any_AKI'] == '1':  # If the new patient has AKI
    diagnosed = True  # Update the condition

  row_list.append(row)  # Append the patient into the list

  previous_subject_id = current_subject_id  # Before moving to another row, update the patient ID

strictly_selected_patients = pd.DataFrame(row_list)  # Create a new dataframe based on the extracted patients
strictly_selected_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,"41401, 42832, 5180, 4280, 4139, 4400, 4019, 34...",0
2,10002155,20345487,32358465,2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576,"1628, 486, 51884, 5781, 2851, 2761, 49121, 414...",0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0
...,...,...,...,...,...,...,...,...
31664,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794,"5780, 5723, 2761, 5771, 5712, 45621, 2768, 303...",0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0
31667,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,"486, 5849, 2760, 5070, 33182, 29410, 78720, 58...",1


In [12]:
strictly_selected_patients = strictly_selected_patients[strictly_selected_patients['los'] >= 2]  # Retain patients who stays longer than 2 days in the ICU

counts = strictly_selected_patients['any_AKI'].value_counts()  # Count the AKI population
print(counts)  # Print counts

any_AKI
0    10476
1     7244
Name: count, dtype: int64


## Loose case

Subsequent diagnosis without AKI are included

In [13]:
row_list = []

diagnosed = False
previous_subject_id = 0
previous_admission_id = 0

for index, row in sepsis_affected_ICU_patients_and_comorbidity_df.iterrows():

  current_subject_id = row['subject_id']  # Get the current subject ID
  current_admission_id = row['hadm_id']

  if previous_admission_id == current_admission_id and diagnosed:  # Same patient and diagnosed AKI previously.
    continue  # Ignore that admission

  if previous_subject_id == current_subject_id and previous_admission_id != current_admission_id and row['any_AKI'] == '0':  # If current admission id != previous admission id
    previous_admission_id = current_admission_id
    row_list.append(row)
    continue

  if previous_subject_id != current_subject_id:  # Encounter a new patient
    diagnosed = False  # Refresh the diagnosis

  if row['any_AKI'] == '1':  # If the new patient has AKI
    diagnosed = True  # Update the condition

  row_list.append(row)  # Append the patient into the list

  #previous_admission_id = current_admission_id
  previous_subject_id = current_subject_id  # Before moving to another row, update the patient ID
  previous_admission_id = current_admission_id

loosely_selected_patients = pd.DataFrame(row_list)  # Create a new dataframe based on the extracted patients
loosely_selected_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,"41401, 42832, 5180, 4280, 4139, 4400, 4019, 34...",0
2,10002155,20345487,32358465,2131-03-09 21:33:00,2131-03-10 18:09:21,0.858576,"1628, 486, 51884, 5781, 2851, 2761, 49121, 414...",0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0
...,...,...,...,...,...,...,...,...
31664,19998878,26489544,34403689,2132-09-30 21:55:00,2132-10-01 15:43:11,0.741794,"5780, 5723, 2761, 5771, 5712, 45621, 2768, 303...",0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0
31667,19999625,25304202,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741,"486, 5849, 2760, 5070, 33182, 29410, 78720, 58...",1


In [14]:
loosely_selected_patients = loosely_selected_patients[loosely_selected_patients['los'] >= 2]  # Retain patients who stays longer than 2 days in the ICU

counts = loosely_selected_patients['any_AKI'].value_counts()  # Count the AKI population
print(counts)  # Print counts

any_AKI
0    11257
1     8397
Name: count, dtype: int64


# Filter Sepsis Affected Population Using Serum Creatinine and Urine Output

## Updated query

In [15]:
import time
updated_query = """

WITH sp AS (  /* All Sepsis Affected Patients */
  SELECT
      i.subject_id,   /* Find distinct count on the admission id */
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
  FROM
      `physionet-data.mimiciv_icu.icustays` AS i
  JOIN
      `physionet-data.mimiciv_derived.sepsis3` as s
  ON
      i.stay_id = s.stay_id AND i.subject_id = s.subject_id
  JOIN
      `physionet-data.mimiciv_hosp.patients` AS p /* Patient demographics */
  ON
      i.subject_id = p.subject_id
  WHERE
      s.sepsis3 = true AND p.anchor_age >= 18 AND p.anchor_age < 90
  GROUP BY
      i.subject_id,
      i.hadm_id,
      i.stay_id,
      i.intime,
      i.outtime,
      i.los
)  /* This table contains all the patients who are affected by Sepsis*/


"""

## Create dictionaries to store data

In [16]:
strictly_selected_patient_serum_creatinine_history = dict()  # Key is a tuple. Content is a pandas dataframe | (subject_id, hadm_id) -> df
strictly_selected_patient_urine_output_history = dict()  # Key is a tuple. Content is a pandas dataframe | (subject_id, hadm_id) -> df
strictly_selected_patient_icu_stay = dict()  # Key is a tuple. Contain a set of icu_stay_ids | (subject_id, hadm_id) -> [icustay_id_1, icu_stay_id_2...]
strict_icu_stay_length = dict() # Key is the icustay_id | icu_stay_id -> (inttime, outtime)

In [17]:
loosely_selected_patient_serum_creatinine_history = dict()  # Key is a tuple. Content is a pandas dataframe | (subject_id, hadm_id) -> df
loosely_selected_patient_urine_output_history = dict()  # Key is a tuple. Content is a pandas dataframe | (subject_id, hadm_id) -> df
loosely_selected_patient_icu_stay = dict()  # Key is a tuple. Contain a set of icu_stay_ids | (subject_id, hadm_id) -> [icustay_id_1, icu_stay_id_2...]
loose_icu_stay_length = dict() # Key is the icustay_id | icu_stay_id -> (inttime, outtime)

# Find all Sepsis affected population Serum creatinine lab tests

## Find all Sepsis affected population serum creatinine

In [18]:
# This query will return all the serum creatinine test for every patient affected by Sepsis
timer_start = time.time()
serum_creatinine_history = run_query(f"""
{updated_query}
SELECT
    sp.subject_id,
    sp.hadm_id,
    sp.stay_id,
    l.charttime,
    l.valuenum,
    l.valueuom
FROM
    sp
JOIN
    `physionet-data.mimiciv_hosp.labevents` AS l
ON
    sp.subject_id = l.subject_id
WHERE
    l.itemid = 50912 OR l.itemid = 52546
ORDER BY
    sp.hadm_id, l.charttime, sp.subject_id;

""")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this query")
serum_creatinine_history

Takes 164.39s to run this query


,subject_id,hadm_id,stay_id,charttime,valuenum,valueuom
0,14990224,20000147,30503572,2121-08-30 16:30:00,0.6,mg/dL
1,14990224,20000147,30503572,2121-08-30 22:27:00,0.7,mg/dL
2,14990224,20000147,30503572,2121-08-31 04:20:00,0.7,mg/dL
3,14990224,20000147,30503572,2121-08-31 15:35:00,0.6,mg/dL
4,14990224,20000147,30503572,2121-09-01 05:28:00,0.8,mg/dL
...,...,...,...,...,...,...
2194056,10355856,29999625,36975675,2157-12-16 07:41:00,1.1,mg/dL
2194057,10355856,29999625,36975675,2157-12-17 08:10:00,1.1,mg/dL
2194058,10355856,29999625,36975675,2157-12-18 05:15:00,1.1,mg/dL
2194059,10355856,29999625,36975675,2157-12-19 05:25:00,1.0,mg/dL


## Find all Sepsis affected population Urine Output

In [19]:
# This query will return all the Urine output during an ICU stay
timer_start = time.time()
urine_output_history = run_query(f"""
  {updated_query}
  SELECT
      sp.subject_id,
      sp.hadm_id,
      sp.stay_id,
      uo.charttime,
      uo.uo_mlkghr_6hr
  FROM
      sp
  JOIN
      `physionet-data.mimiciv_derived.urine_output_rate` AS uo
  ON
      sp.stay_id = uo.stay_id
  ORDER BY
      sp.hadm_id, uo.charttime, sp.subject_id;

""")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this query")
urine_output_history

Takes 160.43s to run this query


,subject_id,hadm_id,stay_id,charttime,uo_mlkghr_6hr
0,14990224,20000147,30503572,2121-08-30 22:30:00,None
1,14990224,20000147,30503572,2121-08-30 23:00:00,None
2,14990224,20000147,30503572,2121-08-31 00:00:00,None
3,14990224,20000147,30503572,2121-08-31 01:00:00,None
4,14990224,20000147,30503572,2121-08-31 02:00:00,None
...,...,...,...,...,...
2337091,10355856,29999625,36975675,2157-11-28 10:50:00,0.370000000
2337092,10355856,29999625,36975675,2157-11-28 16:00:00,None
2337093,10355856,29999625,36975675,2157-11-29 01:00:00,0.591500000
2337094,10355856,29999625,36975675,2157-11-29 06:00:00,0.789800000


# Filter intended population

## Serum creatinine & urine output history history

In [20]:
timer_start = time.time()
case_len = len(strictly_selected_patients)
i = 0
for index, row in strictly_selected_patients.iterrows():
    i = i + 1
    subject_id = row['subject_id']
    admission_id = row['hadm_id']

    # Key to query ICU stay length
    icu_key = row['stay_id']

    # Key to query serum creatinine test history and ICU stay ID
    patient_key = (subject_id, admission_id)

    # Document ICU stay length
    strict_icu_stay_length[icu_key] = (row['intime'], row['outtime'])

    # Document patient ICU stay history
    if patient_key not in strictly_selected_patient_icu_stay:
        strictly_selected_patient_icu_stay[patient_key] = [icu_key]  # Initialize with a new set
    else:
        if icu_key not in strictly_selected_patient_icu_stay[patient_key]:
          strictly_selected_patient_icu_stay[patient_key].append(icu_key)  # Append to the existing list

    # Derive patient serum creatinine history
    if patient_key not in strictly_selected_patient_serum_creatinine_history:
      chunk = serum_creatinine_history[(serum_creatinine_history['subject_id'] == subject_id) &
          (serum_creatinine_history['hadm_id'] == admission_id)
          ][['charttime', 'valuenum', 'valueuom']]  # Find the chunk belonging to a specific subject_id and hadm_id
      strictly_selected_patient_serum_creatinine_history[patient_key] = chunk

    # Derive patient urine output history
    if patient_key not in strictly_selected_patient_urine_output_history:
      chunk = urine_output_history[(urine_output_history['subject_id'] == subject_id) &
          (urine_output_history['hadm_id'] == admission_id)
          ][['charttime', 'uo_mlkghr_6hr']]  # Find the chunk belonging to a specific subject_id and hadm_id
      strictly_selected_patient_urine_output_history[patient_key] = chunk

    if i % 1000 == 0:
      print(f"{round(i/case_len, 2) * 100} % complete... ")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this cell")

6.0 % complete... 
11.0 % complete... 
17.0 % complete... 
23.0 % complete... 
28.000000000000004 % complete... 
34.0 % complete... 
40.0 % complete... 
45.0 % complete... 
51.0 % complete... 
56.00000000000001 % complete... 
62.0 % complete... 
68.0 % complete... 
73.0 % complete... 
79.0 % complete... 
85.0 % complete... 
90.0 % complete... 
96.0 % complete... 
Takes 395.3s to run this cell


In [21]:
timer_start = time.time()
case_len = len(loosely_selected_patients)
i = 0
for index, row in loosely_selected_patients.iterrows():
    i = i + 1
    subject_id = row['subject_id']
    admission_id = row['hadm_id']

    # Key to query ICU stay length
    icu_key = row['stay_id']

    # Key to query serum creatinine test history and ICU stay ID
    patient_key = (subject_id, admission_id)

    # Document ICU stay length
    loose_icu_stay_length[icu_key] = (row['intime'], row['outtime'])

    # Document patient ICU stay history
    if patient_key not in loosely_selected_patient_icu_stay:
        loosely_selected_patient_icu_stay[patient_key] = [icu_key]  # Initialize with a new set
    else:
        if icu_key not in loosely_selected_patient_icu_stay[patient_key]:
          loosely_selected_patient_icu_stay[patient_key].append(icu_key)  # Append to the existing list

    # Derive patient serum creatinine history
    if patient_key not in loosely_selected_patient_serum_creatinine_history:
      chunk = serum_creatinine_history[(serum_creatinine_history['subject_id'] == subject_id) &
          (serum_creatinine_history['hadm_id'] == admission_id)
          ][['charttime', 'valuenum', 'valueuom']]  # Find the chunk belonging to a specific subject_id and hadm_id
      loosely_selected_patient_serum_creatinine_history[patient_key] = chunk

    # Derive patient urine output history
    if patient_key not in loosely_selected_patient_urine_output_history:
      chunk = urine_output_history[(urine_output_history['subject_id'] == subject_id) &
          (urine_output_history['hadm_id'] == admission_id)
          ][['charttime', 'uo_mlkghr_6hr']]  # Find the chunk belonging to a specific subject_id and hadm_id
      loosely_selected_patient_urine_output_history[patient_key] = chunk
    if i % 1000 == 0:
      print(f"{round(i/case_len, 2) * 100} % complete... ")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this cell")

5.0 % complete... 
10.0 % complete... 
15.0 % complete... 
20.0 % complete... 
25.0 % complete... 
31.0 % complete... 
36.0 % complete... 
41.0 % complete... 
46.0 % complete... 
51.0 % complete... 
56.00000000000001 % complete... 
61.0 % complete... 
66.0 % complete... 
71.0 % complete... 
76.0 % complete... 
81.0 % complete... 
86.0 % complete... 
92.0 % complete... 
97.0 % complete... 
Takes 441.17s to run this cell


In [22]:
# Check data length
# strictly_selected_patient_serum_creatinine_history and strictly_selected_patient_icu_stay should be of the same length
# strict_icu_stay_length should be the same length as the strictly selected patient dataframe
print(len(strictly_selected_patient_serum_creatinine_history))
print(len(strictly_selected_patient_urine_output_history))
print(len(strictly_selected_patient_icu_stay))
print(len(strict_icu_stay_length))

17370
17370
17370
17720


In [23]:
# Check data length
# Same layoutas the previous ones
print(len(loosely_selected_patient_serum_creatinine_history))
print(len(loosely_selected_patient_urine_output_history))
print(len(loosely_selected_patient_icu_stay))
print(len(loose_icu_stay_length))

19304
19304
19304
19654


## Some tests

In [24]:
# Test_1
print(strictly_selected_patient_serum_creatinine_history[(16904137, 22747535)].head(100))
print('======================================')
print(strictly_selected_patient_urine_output_history[(16904137, 22747535)].head(100))

print(strictly_selected_patient_icu_stay[(16904137, 22747535)])
print(strict_icu_stay_length[strictly_selected_patient_icu_stay[(16904137, 22747535)][0]])

                 charttime  valuenum valueuom
598256 2105-10-04 17:27:00       1.1    mg/dL
598257 2105-10-05 01:41:00       1.0    mg/dL
598258 2105-10-05 07:47:00       1.0    mg/dL
598259 2105-10-05 14:01:00       1.0    mg/dL
598260 2105-10-06 02:02:00       1.0    mg/dL
598261 2105-10-07 05:40:00       1.5    mg/dL
598262 2105-10-08 05:01:00       2.0    mg/dL
598263 2105-10-09 05:00:00       1.5    mg/dL
598264 2105-10-10 05:42:00       1.2    mg/dL
598265 2105-10-11 05:30:00       1.0    mg/dL
598266 2105-11-30 13:35:00       1.5    mg/dL
598267 2106-02-08 10:35:00       1.9    mg/dL
598268 2106-02-08 13:55:00       1.8    mg/dL
598269 2106-05-10 10:43:00       1.5    mg/dL
598270 2106-08-03 12:18:00       1.6    mg/dL
598271 2106-11-02 14:25:00       2.1    mg/dL
598272 2107-02-15 09:00:00       1.6    mg/dL
598273 2108-04-30 12:17:00       1.5    mg/dL
598274 2108-09-01 13:23:00       1.5    mg/dL
598275 2109-02-16 11:57:00       1.6    mg/dL
598276 2109-08-17 11:31:00       1

In [25]:
# Test 2
print(strictly_selected_patient_serum_creatinine_history[(13746897, 29990494)].head(30))
print('======================================')
print(strictly_selected_patient_urine_output_history[(13746897, 29990494)].head(30))
print(strictly_selected_patient_icu_stay[(13746897, 29990494)])
print(strict_icu_stay_length[strictly_selected_patient_icu_stay[(13746897, 29990494)][0]])

                  charttime  valuenum valueuom
2192099 2162-03-08 05:20:00       0.8    mg/dL
2192100 2162-03-09 05:15:00       0.9    mg/dL
2192101 2162-03-11 06:53:00       1.0    mg/dL
2192102 2162-11-05 22:45:00       1.0    mg/dL
2192103 2162-11-19 20:46:00       0.9    mg/dL
2192104 2163-07-09 21:15:00       0.9    mg/dL
2192105 2163-10-29 18:30:00       0.9    mg/dL
2192106 2164-03-16 16:20:00       1.0    mg/dL
2192107 2164-03-19 09:29:00       1.1    mg/dL
2192108 2164-09-28 07:07:00       0.9    mg/dL
2192109 2164-09-29 06:20:00       1.0    mg/dL
2192110 2164-09-30 07:00:00       0.9    mg/dL
2192111 2165-11-20 10:15:00       0.8    mg/dL
2192112 2166-04-05 23:20:00       0.9    mg/dL
2192113 2166-04-06 06:35:00       0.8    mg/dL
2192114 2166-06-05 16:52:00       1.0    mg/dL
2192115 2166-06-06 07:00:00       0.9    mg/dL
2192116 2166-07-22 14:30:00       1.1    mg/dL
2192117 2166-07-23 07:15:00       0.8    mg/dL
2192118 2166-07-23 13:50:00       0.7    mg/dL
2192119 2166-

In [26]:
# Given start and end timestamps
start_time = strict_icu_stay_length[strictly_selected_patient_icu_stay[(13746897, 29990494)][0]][0]
end_time = strict_icu_stay_length[strictly_selected_patient_icu_stay[(13746897, 29990494)][0]][1]

# Filter the DataFrame to include only rows where 'charttime' is within the specified range
filtered_data = strictly_selected_patient_serum_creatinine_history[(13746897, 29990494)][
    (strictly_selected_patient_serum_creatinine_history[(13746897, 29990494)]['charttime'] <= start_time) &
    (strictly_selected_patient_serum_creatinine_history[(13746897, 29990494)]['charttime'] <= end_time)
]

# Display the filtered data
print(filtered_data)

                  charttime  valuenum valueuom
2192099 2162-03-08 05:20:00       0.8    mg/dL
2192100 2162-03-09 05:15:00       0.9    mg/dL
2192101 2162-03-11 06:53:00       1.0    mg/dL
2192102 2162-11-05 22:45:00       1.0    mg/dL
2192103 2162-11-19 20:46:00       0.9    mg/dL
2192104 2163-07-09 21:15:00       0.9    mg/dL
2192105 2163-10-29 18:30:00       0.9    mg/dL
2192106 2164-03-16 16:20:00       1.0    mg/dL
2192107 2164-03-19 09:29:00       1.1    mg/dL
2192108 2164-09-28 07:07:00       0.9    mg/dL
2192109 2164-09-29 06:20:00       1.0    mg/dL
2192110 2164-09-30 07:00:00       0.9    mg/dL
2192111 2165-11-20 10:15:00       0.8    mg/dL
2192112 2166-04-05 23:20:00       0.9    mg/dL
2192113 2166-04-06 06:35:00       0.8    mg/dL
2192114 2166-06-05 16:52:00       1.0    mg/dL
2192115 2166-06-06 07:00:00       0.9    mg/dL
2192116 2166-07-22 14:30:00       1.1    mg/dL
2192117 2166-07-23 07:15:00       0.8    mg/dL
2192118 2166-07-23 13:50:00       0.7    mg/dL
2192119 2166-

## Check serum and urine output creatinine

Using KDIGO standard to check for AKI for Sepsis-affected patients
* x1.5 baseline
* Urine output < 0.5 mL/kg/6hr

**Baseline creatinine is either the minimum value of the past 7-day measurements or the first measurent after ICU admission**

### strict and loose case

In [27]:
def strict_selection_serum_creatinine(patient_key):
  ICU_admission_time = strict_icu_stay_length[strictly_selected_patient_icu_stay[patient_key][0]][0]  # Start time of that ICU admission
  pre_ICU_admission_time_7_days = ICU_admission_time - pd.Timedelta(days=7)  # ICU admission time 7 days prior to admission
  post_ICU_admission_time_24_hours = ICU_admission_time + pd.Timedelta(hours=24)

  ICU_admission_24_hours_serum_history = strictly_selected_patient_serum_creatinine_history[patient_key][
    (strictly_selected_patient_serum_creatinine_history[patient_key]['charttime'] >= ICU_admission_time) &
    (strictly_selected_patient_serum_creatinine_history[patient_key]['charttime'] <= post_ICU_admission_time_24_hours)
  ]  # The serum check history within that ICU stay

  if ICU_admission_24_hours_serum_history.empty:  # If that patient does not have any serum creatine test during the 24 hours admission. Mark as having AKI and drop later
    return 1

  pre_ICU_admission_serum_history_7_days = strictly_selected_patient_serum_creatinine_history[patient_key][
    (strictly_selected_patient_serum_creatinine_history[patient_key]['charttime'] >= pre_ICU_admission_time_7_days) &
    (strictly_selected_patient_serum_creatinine_history[patient_key]['charttime'] < ICU_admission_time)
  ]  # Get serum creatinine data 7 days prior to ICU admission

  # Find the minimum serum creatinine value during this period. The baseline is either the min of Scr 7 days prior to admission or the first ICU admission
  if pre_ICU_admission_serum_history_7_days.empty:
    creatinine_threshold = ICU_admission_24_hours_serum_history['valuenum'].iloc[0] * 1.5
  else:
    creatinine_threshold = pre_ICU_admission_serum_history_7_days['valuenum'].min() * 1.5

  # Check if there are any values in the 24-hour ICU serum history that are 1.5 times the baseline
  elevated_creatinine = ICU_admission_24_hours_serum_history[
    ICU_admission_24_hours_serum_history['valuenum'] >= creatinine_threshold
  ]

  # Check if there are any such values
  if not elevated_creatinine.empty:
    return 1  # Diagnosed with AKI by KDIGO criteria within 24 hours of admission time.
  else:
    return 0  # Not diagnosed within the time


In [28]:
def loose_selection_serum_creatinine(patient_key):
  ICU_admission_time = loose_icu_stay_length[loosely_selected_patient_icu_stay[patient_key][0]][0]  # Start time of that ICU admission
  pre_ICU_admission_time_7_days = ICU_admission_time - pd.Timedelta(days=7)  # ICU admission time 7 days prior to admission
  post_ICU_admission_time_24_hours = ICU_admission_time + pd.Timedelta(hours=24)

  ICU_admission_24_hours_serum_history = loosely_selected_patient_serum_creatinine_history[patient_key][
    (loosely_selected_patient_serum_creatinine_history[patient_key]['charttime'] >= ICU_admission_time) &
    (loosely_selected_patient_serum_creatinine_history[patient_key]['charttime'] <= post_ICU_admission_time_24_hours)
  ]  # The serum check history within that ICU stay

  if ICU_admission_24_hours_serum_history.empty:  # If that patient does not have any serum creatine test during the 24 hours admission. Mark as having AKI and drop later
    return 1

  pre_ICU_admission_serum_history_7_days = loosely_selected_patient_serum_creatinine_history[patient_key][
    (loosely_selected_patient_serum_creatinine_history[patient_key]['charttime'] >= pre_ICU_admission_time_7_days) &
    (loosely_selected_patient_serum_creatinine_history[patient_key]['charttime'] < ICU_admission_time)
  ]  # Get serum creatinine data 7 days prior to ICU admission

  # Find the minimum serum creatinine value during this period. The baseline is either the min of Scr 7 days prior to admission or the first ICU admission
  if pre_ICU_admission_serum_history_7_days.empty:
    creatinine_threshold = ICU_admission_24_hours_serum_history['valuenum'].iloc[0] * 1.5
  else:
    creatinine_threshold = pre_ICU_admission_serum_history_7_days['valuenum'].min() * 1.5

  # Check if there are any values in the 24-hour ICU serum history that are 1.5 times the baseline
  elevated_creatinine = ICU_admission_24_hours_serum_history[
    ICU_admission_24_hours_serum_history['valuenum'] >= creatinine_threshold
  ]

  # Check if there are any such values
  if not elevated_creatinine.empty:
    return 1  # Diagnosed with AKI by KDIGO criteria within 24 hours of admission time.
  else:
    return 0  # Not diagnosed within the time

In [29]:
def strict_selection_urine_output(patient_key):
  ICU_admission_time = strict_icu_stay_length[strictly_selected_patient_icu_stay[patient_key][0]][0]  # Start time of that ICU admission
  pre_ICU_admission_time_7_days = ICU_admission_time - pd.Timedelta(days=7)  # ICU admission time 7 days prior to admission
  post_ICU_admission_time_24_hours = ICU_admission_time + pd.Timedelta(hours=24)

  ICU_admission_24_hours_urine_history = strictly_selected_patient_urine_output_history[patient_key][
    (strictly_selected_patient_urine_output_history[patient_key]['charttime'] >= ICU_admission_time) &
    (strictly_selected_patient_urine_output_history[patient_key]['charttime'] <= post_ICU_admission_time_24_hours)
  ]  # The urine output check history within that ICU stay

  if ICU_admission_24_hours_urine_history.empty:  # If that patient does not have any urine output during the 24 hours admission. Mark as having AKI and then drop later
    return 0

  # Check if there are any values in 6-hour averaged ICU urine output less than the threshold. Threshold 0.5
  # If the average of urine output is less than 0.5. meaning the patient problem has a urine output value less than 0.5 every hour in that 6 hour span.
  abnormal_urine_output = ICU_admission_24_hours_urine_history[
   ICU_admission_24_hours_urine_history['uo_mlkghr_6hr'] < 0.5
  ]

  # Check if there are any such values
  if not abnormal_urine_output.empty:  # If the returned dataframe containing urine output < 0.5 is not empty
    return 1  # Diagnosed with AKI by KDIGO criteria within 24 hours of admission time.
  else:
    return 0  # Not diagnosed within the time

In [30]:
def loose_selection_urine_output(patient_key):
  ICU_admission_time = loose_icu_stay_length[loosely_selected_patient_icu_stay[patient_key][0]][0]  # Start time of that ICU admission
  post_ICU_admission_time_24_hours = ICU_admission_time + pd.Timedelta(hours=24)  # 24 hours after admission

  ICU_admission_24_hours_urine_history = loosely_selected_patient_urine_output_history[patient_key][
    (loosely_selected_patient_urine_output_history[patient_key]['charttime'] >= ICU_admission_time) &
    (loosely_selected_patient_urine_output_history[patient_key]['charttime'] <= post_ICU_admission_time_24_hours)
  ]  # The urine output check history within that ICU stay

  if ICU_admission_24_hours_urine_history.empty:  # If that patient does not have any urine output during the 24 hours admission. Mark as having AKI and then drop later
    return 0

  # Check if there are any values in 6-hour averaged ICU urine output less than the threshold. Threshold 0.5
  # If the average of urine output is less than 0.5. meaning the patient problem has a urine output value less than 0.5 every hour in that 6 hour span.
  abnormal_urine_output = ICU_admission_24_hours_urine_history[
   ICU_admission_24_hours_urine_history['uo_mlkghr_6hr'] < 0.5
  ]

  # Check if there are any such values
  if not abnormal_urine_output.empty:  # If the returned dataframe containing urine output < 0.5 is not empty
    return 1  # Diagnosed with AKI by KDIGO criteria within 24 hours of admission time.
  else:
    return 0  # Not diagnosed within the time

### Selection function

In [31]:
def check_serum_24h_within_admission(patient_key, selection_type):  # This function checks if a patient is diagnosed with AKI by KDIGO serum creatinine standard
  if selection_type == 'strict':
    return strict_selection_serum_creatinine(patient_key)
  else:
    return loose_selection_serum_creatinine(patient_key)

def check_urine_24h_within_admission(patient_key, selection_type):  # This function checks if a patient is diagnosed with AKI by KDIGO serum creatinine standard
  if selection_type == 'strict':
    return strict_selection_urine_output(patient_key)
  else:
    return loose_selection_urine_output(patient_key)

### On Strictly selected patients

In [32]:
strictly_selected_patients_filtered = strictly_selected_patients.copy()
strictly_selected_patients_filtered

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0
5,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,"C7931, G935, G936, G911, C3490, F05, I10, F172...",0
6,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843,"51881, 2764, 00845, 29281, 5119, 1122, 5781, 2...",0
...,...,...,...,...,...,...,...,...
31660,19998591,24349193,36794489,2185-07-03 22:45:00,2185-07-12 13:40:34,8.621921,"5070, 5849, 2761, 2639, 42832, 514, 3313, 5990...",1
31663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646,"80325, 3485, 8708, E8150, E8495, 87344, V667",0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0


In [33]:
# Iterate over the DataFrame using iterrows and apply the function to each row
results_serum = []
results_urine = []
for index, row in strictly_selected_patients_filtered.iterrows():
    patient_key = (row['subject_id'], row['hadm_id'])
    aki_serum_diagnosis = check_serum_24h_within_admission(patient_key,'strict')
    aki_urine_diagnosis = check_urine_24h_within_admission(patient_key, 'strict')
    results_serum.append(aki_serum_diagnosis)
    results_urine.append(aki_urine_diagnosis)

# Add the results as a new column in the DataFrame
strictly_selected_patients_filtered['serum_check_within_24h'] = results_serum
strictly_selected_patients_filtered['urine_check_within_24h'] = results_urine
# Display the updated DataFrame
strictly_selected_patients_filtered

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI,serum_check_within_24h,urine_check_within_24h
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0,0,0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0,0,1
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0,0,0
5,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,"C7931, G935, G936, G911, C3490, F05, I10, F172...",0,0,0
6,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843,"51881, 2764, 00845, 29281, 5119, 1122, 5781, 2...",0,0,1
...,...,...,...,...,...,...,...,...,...,...
31660,19998591,24349193,36794489,2185-07-03 22:45:00,2185-07-12 13:40:34,8.621921,"5070, 5849, 2761, 2639, 42832, 514, 3313, 5990...",1,0,0
31663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646,"80325, 3485, 8708, E8150, E8495, 87344, V667",0,0,0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1,0,1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0,0,0


In [34]:
# Drop rows where 'AKI_within_24h' is 1
strictly_selected_patients_filtered = strictly_selected_patients_filtered[
    (strictly_selected_patients_filtered['serum_check_within_24h'] != 1) &
    (strictly_selected_patients_filtered['urine_check_within_24h'] != 1)]  # Apply both criteria

# Display the updated DataFrame
print(len(strictly_selected_patients_filtered))

6993


In [35]:
strictly_selected_patients_filtered

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI,serum_check_within_24h,urine_check_within_24h
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0,0,0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0,0,0
5,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,"C7931, G935, G936, G911, C3490, F05, I10, F172...",0,0,0
7,10002428,23473524,35479615,2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,"03843, 51881, 42843, 5990, 00845, 99591, 4280,...",0,0,0
8,10002428,28662225,38875437,2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894,"78097, E9478, 4940, 56210, 53550, 5768, 7904, ...",1,0,0
...,...,...,...,...,...,...,...,...,...,...
31657,19998330,21135114,31417783,2178-10-21 17:20:00,2178-10-23 17:43:28,2.016296,"51881, 42833, 49121, 40390, 5853, 4280, 25002,...",0,0,0
31659,19998330,24492004,32641669,2178-10-01 08:51:00,2178-10-03 23:25:08,2.607037,"51881, 42833, 486, 5849, 49121, 4280, 40390, 5...",1,0,0
31660,19998591,24349193,36794489,2185-07-03 22:45:00,2185-07-12 13:40:34,8.621921,"5070, 5849, 2761, 2639, 42832, 514, 3313, 5990...",1,0,0
31663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646,"80325, 3485, 8708, E8150, E8495, 87344, V667",0,0,0


### On loosely selected patients

In [36]:
loosely_selected_patients_filtered = loosely_selected_patients.copy()
loosely_selected_patients_filtered

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0
5,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,"C7931, G935, G936, G911, C3490, F05, I10, F172...",0
6,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843,"51881, 2764, 00845, 29281, 5119, 1122, 5781, 2...",0
...,...,...,...,...,...,...,...,...
31660,19998591,24349193,36794489,2185-07-03 22:45:00,2185-07-12 13:40:34,8.621921,"5070, 5849, 2761, 2639, 42832, 514, 3313, 5990...",1
31663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646,"80325, 3485, 8708, E8150, E8495, 87344, V667",0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0


In [37]:
# Iterate over the DataFrame using iterrows and apply the function to each row
results_serum = []
results_urine = []
for index, row in loosely_selected_patients_filtered.iterrows():
    patient_key = (row['subject_id'], row['hadm_id'])
    aki_serum_diagnosis = check_serum_24h_within_admission(patient_key,'loose')
    aki_urine_diagnosis = check_urine_24h_within_admission(patient_key, 'loose')
    results_serum.append(aki_serum_diagnosis)
    results_urine.append(aki_urine_diagnosis)

# Add the results as a new column in the DataFrame
loosely_selected_patients_filtered['serum_check_within_24h'] = results_serum
loosely_selected_patients_filtered['urine_check_within_24h'] = results_urine
# Display the updated DataFrame
loosely_selected_patients_filtered

,subject_id,hadm_id,stay_id,intime,outtime,los,merged_icd_codes,any_AKI,serum_check_within_24h,urine_check_within_24h
0,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,"Y92239, Z7952, Z825, Z781, Z515, Z66, Z9981, J...",0,0,0
3,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,6.178912,"41011, 486, 42821, 41402, 99672, 7455, 1628, 2...",0,0,1
4,10002155,28994087,31090461,2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,"486, 51881, 1628, 42822, 4280, 79902, 41401, 4...",0,0,0
5,10002348,22725460,32610785,2112-11-30 23:24:00,2112-12-10 18:25:13,9.792512,"C7931, G935, G936, G911, C3490, F05, I10, F172...",0,0,0
6,10002428,20321825,34807493,2156-04-30 21:53:00,2156-05-02 22:27:20,2.023843,"51881, 2764, 00845, 29281, 5119, 1122, 5781, 2...",0,0,1
...,...,...,...,...,...,...,...,...,...,...
31660,19998591,24349193,36794489,2185-07-03 22:45:00,2185-07-12 13:40:34,8.621921,"5070, 5849, 2761, 2639, 42832, 514, 3313, 5990...",1,0,0
31663,19998843,24842066,30988867,2187-02-05 10:12:00,2187-02-08 18:19:39,3.338646,"80325, 3485, 8708, E8150, E8495, 87344, V667",0,0,0
31665,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,7.023715,"5711, 570, 56723, 29181, 78959, 2761, 7907, 27...",1,0,1
31666,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370,"34541, 43491, 431, 3485, V6284, 11284, 5990, 7...",0,0,0


In [38]:
# Drop rows where 'AKI_within_24h' is 1
loosely_selected_patients_filtered = loosely_selected_patients_filtered[
    (loosely_selected_patients_filtered['serum_check_within_24h'] != 1) &
    (loosely_selected_patients_filtered['urine_check_within_24h'] != 1)]

# Display the updated DataFrame
print(len(loosely_selected_patients_filtered))

7732


### Summary on selection

In [39]:
counts = strictly_selected_patients_filtered['any_AKI'].value_counts()  # Count the AKI population
print(counts)  # Print counts

any_AKI
0    4842
1    2151
Name: count, dtype: int64


In [40]:
counts = loosely_selected_patients_filtered['any_AKI'].value_counts()  # Count the AKI population
print(counts)  # Print counts

any_AKI
0    5172
1    2560
Name: count, dtype: int64


# Collect cohort data

## Split cohort

In [41]:
# Define cohort type and then split the cohort based on AKI outcome
cohort_type = 1  # 1 is strictly patients; 2 is loosely selected patients

if cohort_type == 1:
  positive_patients = strictly_selected_patients_filtered[strictly_selected_patients_filtered['any_AKI'] == '1']
  negative_patients = strictly_selected_patients_filtered[strictly_selected_patients_filtered['any_AKI'] == '0']
  icu_stay_length = strict_icu_stay_length
elif cohort_type == 2:
  positive_patients = loosely_selected_patients_filtered[strictly_selected_patients_filtered['any_AKI'] == '1']
  negative_patients = loosely_selected_patients_filtered[strictly_selected_patients_filtered['any_AKI'] == '0']
  icu_stay_length = loose_icu_stay_length

In [42]:
positive_patients_data = dict()
negative_patients_data = dict()

## Demorgraphics

* Gender
* Age
* Weight
* Height
* BMI

In [43]:
gender_age_race_df = run_query(f"""
  {updated_query}
  SELECT
      sp.subject_id,
      sp.hadm_id,
      sp.stay_id,
      i.gender,
      i.admission_age,
      i.race
  FROM
      sp
  JOIN
      `physionet-data.mimiciv_derived.icustay_detail` AS i
  ON
      sp.stay_id = i.stay_id
  ORDER BY
      sp.stay_id

""")

weight_df = run_query(f"""
    {updated_query}
    SELECT
        sp.subject_id,
        sp.hadm_id,
        sp.stay_id,
        f.weight_admit,
    FROM
        sp
    JOIN
        `physionet-data.mimiciv_derived.first_day_weight` AS f
    ON
        sp.stay_id = f.stay_id
    ORDER BY
        sp.stay_id

""")

height_df = run_query(f"""
    {updated_query}
    SELECT
        sp.subject_id,
        sp.hadm_id,
        sp.stay_id,
        h.charttime,
        h.height
    FROM
        sp
    JOIN
        `physionet-data.mimiciv_derived.height` AS h
    ON
        sp.stay_id = h.stay_id
    ORDER BY
        sp.stay_id

""")

In [44]:
timer_start = time.time()
case_len = len(positive_patients)
i = 0

for index, row in positive_patients.iterrows():
  i = i + 1
  stay_id = row['stay_id']

  if stay_id not in positive_patients_data:
    positive_patients_data[stay_id] = dict()

  gender_age_race = gender_age_race_df[gender_age_race_df['stay_id'] == stay_id][['gender', 'admission_age', 'race']]
  weight = weight_df[weight_df['stay_id'] == stay_id][['weight_admit']]
  height = height_df[height_df['stay_id'] == stay_id][['height']]

  if not gender_age_race.empty:
    positive_patients_data[stay_id]['gender'] = gender_age_race.iloc[0]['gender']
    positive_patients_data[stay_id]['age'] = gender_age_race.iloc[0]['admission_age']
    positive_patients_data[stay_id]['race'] = gender_age_race.iloc[0]['race']
  else:
    positive_patients_data[stay_id]['gender'] = np.nan
    positive_patients_data[stay_id]['age'] = np.nan
    positive_patients_data[stay_id]['race'] = np.nan

  if not height.empty:
    positive_patients_data[stay_id]['height'] = float(height.iloc[0]['height'])
  else:
    positive_patients_data[stay_id]['height'] = np.nan

  if not weight.empty:
    positive_patients_data[stay_id]['weight'] = float(weight.iloc[0]['weight_admit'])
  else:
    positive_patients_data[stay_id]['weight'] = np.nan

  positive_patients_data[stay_id]['BMI'] = positive_patients_data[stay_id]['weight'] / ((positive_patients_data[stay_id]['height'] /100 )** 2)
  if i % 1000 == 0:
    print(f"{round(i/case_len, 2) * 100} % complete... ")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this cell")


46.0 % complete... 
93.0 % complete... 
Takes 8.15s to run this cell


In [45]:
timer_start = time.time()
case_len = len(negative_patients)
i = 0

for index, row in negative_patients.iterrows():
    i = i + 1
    stay_id = row['stay_id']

    if stay_id not in negative_patients_data:
        negative_patients_data[stay_id] = dict()

    gender_age_race = gender_age_race_df[gender_age_race_df['stay_id'] == stay_id][['gender', 'admission_age', 'race']]
    weight = weight_df[weight_df['stay_id'] == stay_id][['weight_admit']]
    height = height_df[height_df['stay_id'] == stay_id][['height']]

    if not gender_age_race.empty:
        negative_patients_data[stay_id]['gender'] = gender_age_race.iloc[0]['gender']
        negative_patients_data[stay_id]['age'] = gender_age_race.iloc[0]['admission_age']
        negative_patients_data[stay_id]['race'] = gender_age_race.iloc[0]['race']
    else:
        negative_patients_data[stay_id]['gender'] = np.nan
        negative_patients_data[stay_id]['age'] = np.nan
        negative_patients_data[stay_id]['race'] = np.nan

    if not height.empty:
        negative_patients_data[stay_id]['height'] = float(height.iloc[0]['height'])
    else:
        negative_patients_data[stay_id]['height'] = np.nan

    if not weight.empty:
        negative_patients_data[stay_id]['weight'] = float(weight.iloc[0]['weight_admit'])
    else:
        negative_patients_data[stay_id]['weight'] = np.nan

    if negative_patients_data[stay_id]['height'] and negative_patients_data[stay_id]['weight']:
        negative_patients_data[stay_id]['BMI'] = negative_patients_data[stay_id]['weight'] / ((negative_patients_data[stay_id]['height'] / 100) ** 2)
    else:
        negative_patients_data[stay_id]['BMI'] = np.nan

    if i % 1000 == 0:
      print(f"{round(i/case_len, 2) * 100} % complete... ")
timer_end = time.time()
print(f"Takes {round(timer_end- timer_start, 2)}s to run this cell")

21.0 % complete... 
41.0 % complete... 
62.0 % complete... 
83.0 % complete... 
Takes 17.26s to run this cell


## Collect vital sign
* Heart rate: 220045
* Body temperature: 223762
* SpO2: 220277
* Systolic pressure: 220050
* Diastolic pressure: 220051
* Mean arterial pressure: 220052
* Respiration rate: 220210

In [46]:
vital_sign_dict = {'heart_rate': 220045,
                   'body_temperature': 223762,
                   'SpO2': 220277,
                   'pressure_sys':220050,
                   'pressure_dias':220051,
                   'pressure_mean':220052,
                   'respiration_rate':220210}

In [47]:
import time
for key, value in vital_sign_dict.items():
    print(f'processing {key} ......')
    timer_start = time.time()
    vital_sign_df = run_query(f"""
      {updated_query}
      SELECT
          sp.subject_id,
          sp.hadm_id,
          sp.stay_id,
          c.charttime,
          c.valuenum,
          c.valueuom
      FROM
          sp
      JOIN
          `physionet-data.mimiciv_icu.chartevents` AS c
      ON
          sp.stay_id = c.stay_id
      WHERE
          c.itemid = {value}
      ORDER BY
          sp.hadm_id, c.charttime, sp.subject_id;

  """)
    case_len = len(positive_patients)
    i = 0
    for index, row in positive_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']

      if stay_id not in positive_patients_data:
        positive_patients_data[stay_id] = dict()

      ICU_start_time = icu_stay_length[stay_id][0]
      stop_sampling_time = ICU_start_time + pd.Timedelta(hours=24)

      data_df = vital_sign_df[
          (vital_sign_df['stay_id'] == stay_id) &
          (vital_sign_df['charttime'] >= ICU_start_time) &
          (vital_sign_df['charttime'] <= stop_sampling_time)][['charttime', 'valuenum', 'valueuom']]

      positive_patients_data[stay_id][key] = data_df
      if i % 1000 == 0:
        print(f"Positive patients: {round(i/case_len, 2) * 100} % complete... ")

    case_len = len(negative_patients)
    i = 0
    for index, row in negative_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']

      if stay_id not in negative_patients_data:
        negative_patients_data[stay_id] = dict()

      ICU_start_time = icu_stay_length[stay_id][0]
      stop_sampling_time = ICU_start_time + pd.Timedelta(hours=24)

      data_df = vital_sign_df[
          (vital_sign_df['stay_id'] == stay_id) &
          (vital_sign_df['charttime'] >= ICU_start_time) &
          (vital_sign_df['charttime'] <= stop_sampling_time)][['charttime', 'valuenum', 'valueuom']]

      negative_patients_data[stay_id][key] = data_df
      if i % 1000 == 0:
        print(f"Negative patients: {round(i/case_len, 2) * 100} % complete... ")
    timer_end = time.time()
    print(f'\t took {round(timer_end - timer_start,2)}s')

processing heart_rate ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Negative patients: 21.0 % complete... 
Negative patients: 41.0 % complete... 
Negative patients: 62.0 % complete... 
Negative patients: 83.0 % complete... 
	 took 659.51s
processing body_temperature ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Negative patients: 21.0 % complete... 
Negative patients: 41.0 % complete... 
Negative patients: 62.0 % complete... 
Negative patients: 83.0 % complete... 
	 took 43.73s
processing SpO2 ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Negative patients: 21.0 % complete... 
Negative patients: 41.0 % complete... 
Negative patients: 62.0 % complete... 
Negative patients: 83.0 % complete... 
	 took 662.3s
processing pressure_sys ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Negative patients: 21.0 % complete... 
Negative patients: 4

## Collect lab data

* bilirubin_total: 50885, 53089

* anion_gap: 52500, 50868

* albumin: 52022, 50862, 53085, 53138

* serum_chloride: 52535/50902

* serum_potassium: 50971/52610

* serum_sodium: 50983/52623

* serum_lactate: 50813

* serum_magnesium: 50960

* partial_thromboplastin_time (PTT): 51275/52923

* prothrombin_time (PT): 51274/52921

* blood urea nitrogen (BUN): 51006/52647/51842

* serum_glucose: 220621

* serum_creatinine: 50912/52546

* Hematocrit: 50810/52028/51638/51639

* hemoglobin: 50811

* platelets: 51265

* white blood cell(WBC): 51755/51756

* red blood cell(RBC): 51279/52170/51493

* serum_bicarbonate: 50803/50882

* pH:50820   (Fluid):52041   (Other body fluid):50831
--------------------------------------------------------
**Drop:** serum_calcium:52035/52034/50893

**Drop:** International normalised ration: 50882

**Drop:** pco2:50818   (Fluid):52040  (other body fluid):50830

In [48]:
lab_items_dict = {
    'bilirubin_total': [50885, 53089, 51464],
    'anion_gap': [52500, 50868],
    'albumin': [52022, 50862, 53085, 53138],
    'serum_chloride': [52535, 50902],
    'serum_potassium': [50971, 52610],
    'serum_sodium': [50983, 52623],
    'serum_lactate': [50813],
    'serum_magnesium': [50960],
    'PTT': [51275, 52923],
    'PT': [51274, 52921],
    'BUN': [51006, 52647, 51842],
    'serum_creatinine': [50912, 52546],
    'hematocrit': [50810, 52028, 51638, 51639],
    'hemoglobin': [50811, 51640, 51222],
    'platelets': [51265],
    'WBC': [52219, 51300, 51755, 51756, 51301],
    'RBC': [51279, 52170],
    'serum_bicarbonate': [50803, 50882],
    'pH': [50820, 52041, 50831]
}

In [49]:
for key, value in lab_items_dict.items():
    if len(value) == 1:
        item_ids = f"({value[0]})"  # If there is a single value
    else:
        item_ids = f"({', '.join(map(str, value))})"  # If there are multiple values

    print(f'processing {key} ......')
    timer_start = time.time()
    lab_tests_df = run_query(f"""
      {updated_query}
      SELECT
          sp.subject_id,
          sp.hadm_id,
          sp.stay_id,
          l.itemid,
          l.charttime,
          l.valuenum,
          l.valueuom
      FROM
          sp
      JOIN
          `physionet-data.mimiciv_hosp.labevents` AS l
      ON
          sp.subject_id = l.subject_id AND sp.hadm_id = l.hadm_id

      WHERE
          l.itemid in {item_ids}
      ORDER BY
          sp.stay_id, l.charttime

    """)
    case_len = len(positive_patients)
    i = 0
    for index, row in positive_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']

      if stay_id not in positive_patients_data:
        positive_patients_data[stay_id] = dict()

      ICU_start_time = icu_stay_length[stay_id][0]
      stop_sampling_time = ICU_start_time + pd.Timedelta(hours=24)

      data_df = lab_tests_df[
          (lab_tests_df['stay_id'] == stay_id) &
          (lab_tests_df['charttime'] >= ICU_start_time) &
          (lab_tests_df['charttime'] <= stop_sampling_time)][['charttime', 'valuenum', 'valueuom']]

      positive_patients_data[stay_id][key] = data_df
      if i % 1000 == 0:
        print(f"Positive patients: {round(i/case_len, 2) * 100} % complete... ")

    case_len = len(negative_patients)
    i = 0
    for index, row in negative_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']

      if stay_id not in negative_patients_data:
        negative_patients_data[stay_id] = dict()

      ICU_start_time = icu_stay_length[stay_id][0]
      stop_sampling_time = ICU_start_time + pd.Timedelta(hours=24)

      data_df = lab_tests_df[
          (lab_tests_df['stay_id'] == stay_id) &
          (lab_tests_df['charttime'] >= ICU_start_time) &
          (lab_tests_df['charttime'] <= stop_sampling_time)][['charttime', 'valuenum', 'valueuom']]

      negative_patients_data[stay_id][key] = data_df
      if i % 1000 == 0:
        print(f"Positive patients: {round(i/case_len, 2) * 100} % complete... ")

    timer_end = time.time()
    print(f'\t took {round(timer_end - timer_start,2)}s')


processing bilirubin_total ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 53.91s
processing anion_gap ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 108.36s
processing albumin ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 29.67s
processing serum_chloride ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients

In [50]:
for key in positive_patients_data.keys():
  print(positive_patients_data[key]['hemoglobin'])

Streaming output truncated to the last 5000 lines.
178999 2160-09-07 17:44:00      10.7     g/dL
179000 2160-09-07 21:40:00       9.9     g/dL
179001 2160-09-08 02:06:00       9.4     g/dL
179002 2160-09-08 02:29:00       9.7     g/dL
                 charttime  valuenum valueuom
179405 2114-07-01 03:59:00      13.0     g/dL
179406 2114-07-02 01:18:00      11.0     g/dL
                charttime  valuenum valueuom
39954 2136-07-05 17:06:00      15.5     g/dL
39955 2136-07-05 18:40:00      15.7     g/dL
39956 2136-07-06 02:41:00      15.3     g/dL
39957 2136-07-06 07:30:00      14.7     g/dL
                 charttime  valuenum valueuom
189216 2169-02-09 03:10:00       7.7     g/dL
                 charttime  valuenum valueuom
255612 2153-02-27 04:29:00      10.8     g/dL
                charttime  valuenum valueuom
47749 2139-11-22 09:11:00      12.9     g/dL
47750 2139-11-23 02:46:00      12.2     g/dL
                 charttime  valuenum valueuom
438909 2153-11-11 13:46:00       9.9 

## Comorbidities

* congestive heart failure
* hypertension
* chronic pulmonary disease
* diabetes
* liver disease
* peptic ulcer disease
* peripheral vascular disease
* myocardial infarction
* cerebrovascular disease
* hiv
* chronic kidney disease
* tumor

In [51]:
comorbidities_dict = {
    'CHF':'congestive heart failure',
    'hypertension':'hypertension',
    'CPD':'chronic pulmonary disease',
    'diabetes':'diabetes',
    'liver_disease':'liver disease',
    'PUD':'peptic ulcer disease',
    'PVD':'peripheral vascular disease',
    'MI':'myocardial infarction',
    'CD':'cerebrovascular disease',
    'hiv':'hiv',
    'CDK':'chronic kidney disease',
    'cancer':'cancer carcinoma neoplasm malignancy tumor sarcoma lymphoma leukemia myeloma melanoma blastoma metastasis metastatic'
}

In [52]:
for key, values in comorbidities_dict.items():

    terms = values.split()  # Split the value into individual words
    if not key == 'cancer':
      select_clause = " AND ".join([f"LOWER(long_title) LIKE '%{term}%'" for term in terms])  # Concatenate the selection clause
    else:  # Check for any type of cancer
      select_clause = " OR ".join([f"LOWER(long_title) LIKE '%{term}%'" for term in terms])

    print(f'processing {key} ......')
    timer_start = time.time()
    icd_codes_df = run_query(f"""
    SELECT
      icd_code
    FROM
      `physionet-data.mimiciv_hosp.d_icd_diagnoses`
    WHERE
      {select_clause}
    """)
    case_len = len(positive_patients)
    i = 0
    for index, row in positive_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']
      icd_diagnoses_list = row['merged_icd_codes'].split(',')

      if stay_id not in positive_patients_data:
        positive_patients_data[stay_id] = dict()

      exists = icd_codes_df['icd_code'].isin(icd_diagnoses_list).any()
      positive_patients_data[stay_id][key] = 1 if exists else 0
      if i % 1000 == 0:
        print(f"Positive patients: {round(i/case_len, 2) * 100} % complete... ")


    case_len = len(negative_patients)
    i = 0
    for index, row in negative_patients.iterrows():
      i = i + 1
      stay_id = row['stay_id']
      icd_diagnoses_list = row['merged_icd_codes'].split(',')

      if stay_id not in negative_patients_data:
        negative_patients_data[stay_id] = dict()

      exists = icd_codes_df['icd_code'].isin(icd_diagnoses_list).any()
      negative_patients_data[stay_id][key] = 1 if exists else 0
      if i % 1000 == 0:
        print(f"Positive patients: {round(i/case_len, 2) * 100} % complete... ")

    timer_end = time.time()
    print(f'\t took {round(timer_end - timer_start,2)}s')

processing CHF ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 2.26s
processing hypertension ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 1.93s
processing CPD ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
Positive patients: 62.0 % complete... 
Positive patients: 83.0 % complete... 
	 took 1.82s
processing diabetes ......
Positive patients: 46.0 % complete... 
Positive patients: 93.0 % complete... 
Positive patients: 21.0 % complete... 
Positive patients: 41.0 % complete... 
P

In [53]:
for key in positive_patients_data.keys():
  print(positive_patients_data[key].keys())

dict_keys(['gender', 'age', 'race', 'height', 'weight', 'BMI', 'heart_rate', 'body_temperature', 'SpO2', 'pressure_sys', 'pressure_dias', 'pressure_mean', 'respiration_rate', 'bilirubin_total', 'anion_gap', 'albumin', 'serum_chloride', 'serum_potassium', 'serum_sodium', 'serum_lactate', 'serum_magnesium', 'PTT', 'PT', 'BUN', 'serum_creatinine', 'hematocrit', 'hemoglobin', 'platelets', 'WBC', 'RBC', 'serum_bicarbonate', 'pH', 'CHF', 'hypertension', 'CPD', 'diabetes', 'liver_disease', 'PUD', 'PVD', 'MI', 'CD', 'hiv', 'CDK', 'cancer'])
dict_keys(['gender', 'age', 'race', 'height', 'weight', 'BMI', 'heart_rate', 'body_temperature', 'SpO2', 'pressure_sys', 'pressure_dias', 'pressure_mean', 'respiration_rate', 'bilirubin_total', 'anion_gap', 'albumin', 'serum_chloride', 'serum_potassium', 'serum_sodium', 'serum_lactate', 'serum_magnesium', 'PTT', 'PT', 'BUN', 'serum_creatinine', 'hematocrit', 'hemoglobin', 'platelets', 'WBC', 'RBC', 'serum_bicarbonate', 'pH', 'CHF', 'hypertension', 'CPD', 'd

In [54]:
for key in negative_patients_data.keys():
  print(negative_patients_data[key].keys())

Streaming output truncated to the last 5000 lines.
dict_keys(['gender', 'age', 'race', 'height', 'weight', 'BMI', 'heart_rate', 'body_temperature', 'SpO2', 'pressure_sys', 'pressure_dias', 'pressure_mean', 'respiration_rate', 'bilirubin_total', 'anion_gap', 'albumin', 'serum_chloride', 'serum_potassium', 'serum_sodium', 'serum_lactate', 'serum_magnesium', 'PTT', 'PT', 'BUN', 'serum_creatinine', 'hematocrit', 'hemoglobin', 'platelets', 'WBC', 'RBC', 'serum_bicarbonate', 'pH', 'CHF', 'hypertension', 'CPD', 'diabetes', 'liver_disease', 'PUD', 'PVD', 'MI', 'CD', 'hiv', 'CDK', 'cancer'])
dict_keys(['gender', 'age', 'race', 'height', 'weight', 'BMI', 'heart_rate', 'body_temperature', 'SpO2', 'pressure_sys', 'pressure_dias', 'pressure_mean', 'respiration_rate', 'bilirubin_total', 'anion_gap', 'albumin', 'serum_chloride', 'serum_potassium', 'serum_sodium', 'serum_lactate', 'serum_magnesium', 'PTT', 'PT', 'BUN', 'serum_creatinine', 'hematocrit', 'hemoglobin', 'platelets', 'WBC', 'RBC', 'serum_b

# Write data into JSON

In [55]:
import json
import numpy as np
import pandas as pd

def convert_complex_types(o):
    # Handle numpy types
    if isinstance(o, (np.int64, np.int32)):
        return int(o)
    if isinstance(o, (np.float64, np.float32)):
        return float(o)
    # Handle pandas DataFrame, converting to a dictionary
    if isinstance(o, pd.DataFrame):
        return o.to_dict(orient='records')
    # Handle pandas Timestamp
    if isinstance(o, pd.Timestamp):
        return o.isoformat()  # Convert Timestamp to ISO format string
    raise TypeError(f"Object of type {type(o)} is not JSON serializable")

# Save positive_patients_data separately
with open('positive_patients_data.json', 'w') as json_file:
    json.dump(positive_patients_data, json_file, default=convert_complex_types, indent=4)

# Save negative_patients_data separately
with open('negative_patients_data.json', 'w') as json_file:
    json.dump(negative_patients_data, json_file, default=convert_complex_types, indent=4)

print("Positive and negative patients data stored separately as JSON files.")


Positive and negative patients data stored separately as JSON files.


In [56]:
# Download the dataset
from google.colab import files
# Download the positive_patients_data.json file
files.download('positive_patients_data.json')
# Download the negative_patients_data.json file
files.download('negative_patients_data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>